<img src='http://www-scf.usc.edu/~ghasemig/images/sharif.png' alt="SUT logo" width=500 height=300 align=center class="saturate" >

<br>
<font>
<div dir=ltr align=center>
<font color=0F5298 size=7>
    Artificial Intelligence <br>
    Computer Engineering Department <br>
    Spring 2024<br>
<font color=3C99D size=5>
    Practical Assignment 3 - Heart Disease Prediction using Hidden Markov Models  <br>
<font color=696880 size=4>
Omid Daliran


# Personal Data

In [1]:
# Set your student number and name
student_number = 401106266
Name = "Mahdi"
Last_Name = "Ali nejad"

# Libraries

In [2]:
import csv
import numpy as np
import pandas as pd

# Q2: Heart Disease Prediction using Hidden Markov  (100 Points)

# Introduction


In this notebook, we explore the application of Hidden Markov Models (HMM) in predicting heart disease risk based on DNA sequences. Heart disease is a prevalent and life-threatening condition, and early detection is crucial for effective management. By leveraging HMM, we aim to identify regions within DNA sequences associated with high and low GC content, which have been linked to heart disease susceptibility.


# Hidden Markov Models (HMM) (40 points)

Hidden Markov Models are probabilistic models widely used in sequential data analysis, particularly in bioinformatics. They are characterized by states, transitions between states, and emission probabilities associated with each state. In our context, the states represent the underlying biological characteristics of DNA sequences, while the emission probabilities signify the likelihood of observing specific nucleotides given each state.

To facilitate our heart disease classification, we'll implement a custom Hidden Markov Model (HMM) class in Python. This class will include the Viterbi and Forward algorithms.

In [3]:
class HMM:
    def __init__(self, states: list, emissions: list, start_probabilities: list, transition_probabilities: list):
        self.sequence = None
        self.states = states
        self.state_labels = {i: state for i, state in enumerate(states)}
        self.emissions = emissions
        self.start_probabilities = start_probabilities
        self.transition_probabilities = transition_probabilities

    def viterbi(self, sequence: str) -> tuple:
        t = len(sequence)
        self.sequence = sequence
        probs = self.recursive_viterbi(t)
        max_probability = self.get_max_probs(probs)
        return max_probability
    

    def get_max_probs(self, probs):
        max_probability = max(probs.items(), key=lambda x: x[1])
        return max_probability
    

    def recursive_viterbi(self, t):
        if t == 1:
            probs = self.calculate_probability_base_state(self.sequence[t - 1])
            return probs

        probs = self.recursive_viterbi(t - 1)
        new_probs = {}
        for _, state in self.state_labels.items():
            paths = self.cal_paths_to_state(state, probs)
            max_probability = self.get_max_probs(paths)
            new_probs[max_probability[0] + state] = (max_probability[1] *
                                                     self.get_emission_probability(self.sequence[t-1], state))
        
        return new_probs
            
                

    def forward_algorithm(self, sequence: str) -> list:
        t = len(sequence)
        self.sequence = sequence
        probs = self.recursive_forward(t)
        max_probability = self.normalize(probs)

        ans = []
        for _, state in self.state_labels.items():
            ans.append(max_probability[state])
            
        return ans
    
    

    def calculate_probability_base_state(self, evidence):
        dic = {self.state_labels[i]: self.start_probabilities[i] for i in range(len(self.states))}
        for key, value in dic.items():
            dic[key] *= self.get_emission_probability(evidence, key)
        return dic

    def get_emission_probability(self, evidence, key):
        return self.emissions[self.find_state_num(key)][evidence]

    def find_state_num(self, val):
        for key, value in self.state_labels.items():
            if value == val:
                return key
        return None

    def cal_paths_to_state(self, state, probs):
        dic = {}
        for key, value in probs.items():
            dic[key] = value * self.get_transition_probability(key[-1], state)
        return dic

    def get_transition_probability(self, key, state):
        num = self.find_state_num(key)
        return self.transition_probabilities[num][state]

    def recursive_forward(self, t):
        if t == 1:
            probs = self.calculate_probability_base_state(self.sequence[t - 1])
            return probs

        probs = self.recursive_forward(t - 1)
        new_probs = {}
        for _, state in self.state_labels.items():
            paths = self.cal_paths_to_state(state, probs)
            max_probability = self.get_sum_probs(paths)
            new_probs[state] = (max_probability * self.get_emission_probability(self.sequence[t-1], state))
        
        return new_probs

    def get_sum_probs(self, paths):
        max_probability = 0
        for path, val in paths.items():
            max_probability += val
        return max_probability

    def normalize(self, probs):
        m = self.get_sum_probs(probs)
        new_probs = {}
        for key, value in probs.items():
            new_probs[key] = value/m
        
        return new_probs



GC content, the proportion of guanine (G) and cytosine (C) nucleotides in a DNA sequence, is a fundamental genomic feature. Variations in GC content have been implicated in various biological processes, including gene regulation and disease susceptibility. Specifically, high GC content (H) regions are associated with increased gene expression and potential regulatory elements, while low GC content (L) regions may indicate structural elements or regions prone to mutations. The transition and emission and start probabilities are as shown in the image below:

![Probabilities](Probabilities.png)

In [4]:
# TODO: Complete and run this code to check your implementation

states = ['H', 'L']
emissions = [{'A': 0.2, 'C': 0.3, 'G': 0.3, 'T': 0.2}, {'A': 0.3, 'C': 0.2, 'G': 0.2, 'T': 0.3}]
start_probabilities = [0.5, 0.5]
transition_probabilities = [{'H': 0.5, 'L': 0.5}, {'H': 0.4, 'L': 0.6}]

hmm = HMM(states, emissions, start_probabilities, transition_probabilities)

# Example DNA sequence
# sequence = "ATGCGCGATCGATCGAATCGCGT"
sequence = "CTCCAATACCCCCCACAAGAACACACCCATAAAATTGCAACCCACAAATTAACAGCACCCCACAATACTACCAAACAACATAACCGCCCCGCCCGACTAC"
    

# Viterbi algorithm
path, max_probability = hmm.viterbi(sequence)
print("Viterbi Path:", path)
print("Max Probability (Viterbi):", max_probability)

# Forward algorithm
probability = hmm.forward_algorithm(sequence)
print("Probability (Forward Algorithm):", probability)

Viterbi Path: HLHHLLLLHHHHHHLLLLLLLLLLLHHHLLLLLLLLHHLLHHHLLLLLLLLLLLHHLHHHHLLLLLLLLLHHLLLLLLLLLLLHHHHHHHHHHHHLLLLH
Max Probability (Viterbi): 6.61373394607485e-82
Probability (Forward Algorithm): [0.5348975765446967, 0.4651024234553033]


# Calculating Probabilities for Blood Type, Height, and Weight (10 points)

After obtaining the sequence of H and L regions from the DNA using the Viterbi algorithm, we proceed to calculate the probabilities for other attributes such as blood type, height, weight, and the presence or absence of DCM (DCM, or Dilated Cardiomyopathy, is a heart condition characterized by the enlargement of the heart's left ventricle. It can lead to heart failure if not properly managed.) using the Forward algorithm. In this step, we aim to leverage the Hidden Markov Model (HMM) to infer these attributes based on the observed GC content sequence.


We will use the functionality of the Forward algorithm to compute the probabilities associated with each possible state after observing the sequence.

Attributes and Possible States:
* Blood Type: O, A, B, AB
* Height: 160 cm, 170 cm, 180 cm, 190 cm
* Weight: 50 kg, 65 kg, 80 kg, 95 kg
* DCM: Y (indicating presence) or N (indicating absence)


In [5]:
def probabilities_from_sequence(sequence: str) -> tuple:
    '''
    This function, `probabilities_from_sequence(sequence)`, computes the probabilities associated with attributes such as blood type, height, weight, and the presence or absence of Dilated Cardiomyopathy (DCM) based on an input DNA sequence.
  
    ### Input:
    - `sequence`: The observed DNA sequence for which probabilities are to be computed.
  
    ### Output:
    The function returns dictionaries containing the probabilities for each possible state of the respective attributes:
    - `height_probability_dic`: Probabilities for height categories (160 cm, 170 cm, 180 cm, 190 cm).
    - `weight_probability_dic`: Probabilities for weight categories (50 kg, 65 kg, 80 kg, 95 kg).
    - `blood_type_probability_dic`: Probabilities for blood types (O, A, B, AB).
    - `DCM_probability_dic`: Probabilities for the presence (Y) or absence (N) of Dilated Cardiomyopathy (DCM).
    '''
    # Define the emission probabilities for each feature

    height_emissions = [{'H': 0.2, 'L': 0.8}, {'H': 0.3, 'L': 0.7},
                        {'H': 0.9, 'L': 0.1}, {'H': 0.8, 'L': 0.2}]

    weight_emissions = [{'H': 0.3, 'L': 0.7}, {'H': 0.1, 'L': 0.9},
                        {'H': 0.2, 'L': 0.8}, {'H': 0.7, 'L': 0.3}]

    blood_type_emissions = [{'H': 0.75, 'L': 0.25}, {'H': 0.8, 'L': 0.2},
                            {'H': 0.85, 'L': 0.15}, {'H': 0.7, 'L': 0.3}]

    DCM_emissions = [{'H': 0.7, 'L': 0.3}, {'H': 0.6, 'L': 0.4}]

    # Define the transition probabilities for each feature
    height_transition_probabilities = [[0.7, 0.2, 0.05, 0.05], [0.1, 0.8, 0.05, 0.05], [0.1, 0.1, 0.7, 0.1],
                                       [0.1, 0.1, 0.1, 0.7]]
    weight_transition_probabilities = [[0.7, 0.2, 0.05, 0.05], [0.1, 0.8, 0.05, 0.05], [0.1, 0.1, 0.7, 0.1],
                                       [0.1, 0.1, 0.1, 0.7]]
    blood_type_transition_probabilities = [[0.7, 0.2, 0.05, 0.05], [0.1, 0.8, 0.05, 0.05], [0.1, 0.1, 0.7, 0.1],
                                           [0.1, 0.1, 0.1, 0.7]]
    DCM_transition_probabilities = [[0.9, 0.1], [0.1, 0.9]]

    # Define start probabilities for each state
    height_start_probabilities = [0.1, 0.4, 0.4, 0.1]
    weight_start_probabilities = [0.15, 0.35, 0.35, 0.15]
    blood_type_start_probabilities = [0.1, 0.35, 0.35, 0.2]
    DCM_start_probabilities = [0.25, 0.75]
    
    heights = ['6', '7', '8', '9']
    weights = ['5', '6', '8', '9']
    blood_types = ['O', 'A', 'B', 'J']
    DCMs = ['Y', 'N']
    
    # Heights
    tpl = []
    for i in range(4):
        tp = {}
        for j in range(4):
            tp[heights[j]] = height_transition_probabilities[i][j]
        tpl.append(tp)
        
    height_HMM = HMM(heights, height_emissions, height_start_probabilities, tpl)
    
    res = height_HMM.forward_algorithm(sequence)
    height_probability_dic = {}
    for i in range(4):
        h = heights[i]
        h = '1' + h + '0'
        h = int(h)
        height_probability_dic[h] = res[i]
        
        
    # weights
    tpl = []
    for i in range(4):
        tp = {}
        for j in range(4):
            tp[weights[j]] = weight_transition_probabilities[i][j]
        tpl.append(tp)
    
    weight_HMM = HMM(weights, weight_emissions, weight_start_probabilities, tpl)
    
    res = weight_HMM.forward_algorithm(sequence)
    weight_probability_dic = {}
    for i in range(4):
        h = weights[i]
        if i % 2 == 0:
            h += '0'
        else:
            h += '5'
        h = int(h)
        weight_probability_dic[h] = res[i]
    
    
    # blood type
    tpl = []
    for i in range(4):
        tp = {}
        for j in range(4):
            tp[blood_types[j]] = blood_type_transition_probabilities[i][j]
        tpl.append(tp)
        
    blood_HMM = HMM(blood_types, blood_type_emissions, blood_type_start_probabilities, tpl)
    
    res = blood_HMM.forward_algorithm(sequence)
    blood_type_probability_dic = {}
    for i in range(4):
        h = blood_types[i]
        if h == 'J':
            h = 'AB'
            
        blood_type_probability_dic[h] = res[i]
        
        
    # DCM
    tpl = []
    for i in range(2):
        ed = {}
        tp = {}
        for j in range(2):
            ed[DCMs[j]] = DCM_emissions[j]
            tp[DCMs[j]] = DCM_transition_probabilities[i][j]
        tpl.append(tp)
        
    DCM_HMM = HMM(DCMs, DCM_emissions, DCM_start_probabilities, tpl)
    
    res = DCM_HMM.forward_algorithm(sequence)
    DCM_probability_dic = {}
    for i in range(2):
        h = DCMs[i]
        DCM_probability_dic[h] = res[i]
        
    # TODO: complete the code

    return height_probability_dic, weight_probability_dic, blood_type_probability_dic, DCM_probability_dic



In [6]:
# Run this code to check your implementation
# Example sequence of 'H' and 'L'
# sequence = "HHLLHHHLLHLLLLHHLL"
sequence = "HLHHLLLLHHHHHHLLLLLLLLLLLHHHLLLLLLLLHHLLHHHLLLLLLLLLLLHHLHHHHLLLLLLLLLHHLLLLLLLLLLLHHHHHHHHHHHHLLLLH"

height_probability, weight_probability, blood_type_probability, DCM_probability = probabilities_from_sequence(sequence)

print("Probability of height given sequence:", height_probability)
print("Probability of weight given sequence:", weight_probability)
print("Probability of blood type given sequence:", blood_type_probability)
print("Probability of DCM given sequence:", DCM_probability)

Probability of height given sequence: {160: 0.22019458270563988, 170: 0.46938753977303344, 180: 0.15493736960539042, 190: 0.15548050791593632}
Probability of weight given sequence: {50: 0.31987029288646424, 65: 0.22918457097625072, 80: 0.17341681239768114, 95: 0.2775283237396038}
Probability of blood type given sequence: {'O': 0.2753321437518179, 'A': 0.3647542741870535, 'B': 0.12749216617033832, 'AB': 0.2324214158907902}
Probability of DCM given sequence: {'Y': 0.4216714899580485, 'N': 0.5783285100419515}


# Bayesian Network for Heart Disease Probability Calculation (30 points)

In this section, we introduce a Bayesian network to calculate the probability of heart disease based on various features. We assume that the relationships between features follow a specific structure, depicted in the image below.

![Bayesian Net](BN.png)

The Bayesian network comprises nodes representing different features, including sex, weight, height, Dilated Cardiomyopathy (DCM), blood type, and heart disease. The directed edges between nodes indicate probabilistic dependencies between them.

Function Implementation:
We will implement two functions to facilitate the calculation of heart disease probability:

1. calculate_sex_probability:

  This function calculates the probability of sex based on the given probabilities of weight and height.

2. calculate_heart_disease_probability:

  This function computes the probability of heart disease based on the probabilities of sex, DCM, and blood type.

In [7]:
# The probability of sex given height and weight, the tuple is in this order: (sex, height, weight)
P_sex_given_hw = {
    ('M', 160, 50): 0.7, ('F', 160, 50): 0.3,
    ('M', 160, 65): 0.2, ('F', 160, 65): 0.8,
    ('M', 160, 80): 0.3, ('F', 160, 80): 0.7,
    ('M', 160, 95): 0.2, ('F', 160, 95): 0.8,
    ('M', 170, 50): 0.8, ('F', 170, 50): 0.2,
    ('M', 170, 65): 0.75, ('F', 170, 65): 0.25,
    ('M', 170, 80): 0.4, ('F', 170, 80): 0.6,
    ('M', 170, 95): 0.3, ('F', 170, 95): 0.7,
    ('M', 180, 50): 0.9, ('F', 180, 50): 0.1,
    ('M', 180, 65): 0.7, ('F', 180, 65): 0.3,
    ('M', 180, 80): 0.65, ('F', 180, 80): 0.35,
    ('M', 180, 95): 0.4, ('F', 180, 95): 0.6,
    ('M', 190, 50): 0.95, ('F', 190, 50): 0.05,
    ('M', 190, 65): 0.8, ('F', 190, 65): 0.2,
    ('M', 190, 80): 0.6, ('F', 190, 80): 0.4,
    ('M', 190, 95): 0.95, ('F', 190, 95): 0.05
}

In [8]:
def calculate_sex_probability(height_prob: dict, weight_prob: dict) -> dict:
    '''
    This function, `calculate_sex_probability(height_prob, weight_prob)`, computes the probability of sex based on the probabilities of height and weight.

    Parameters:
    - height_prob: Dictionary containing probabilities for height categories.
    - weight_prob: Dictionary containing probabilities for weight categories.

    Returns:
    - Dictionary containing probabilities for male (M) and female (F) sexes.
    '''

    sex_dic = {'M': 0, 'F': 1}

    sex_probability = np.zeros(2)
    
    for h, hval in height_prob.items():
        for w, wval in weight_prob.items():
            for s, sval in sex_dic.items():
                sex_probability[sval] += P_sex_given_hw[(s, h, w)] * hval * wval
        

    # TODO: complete the code

    return {'M': sex_probability[0], 'F': sex_probability[1]}


In [9]:
# Run this code to test you implementation
# Example probabilities of height
# P_height = {160: 0.1, 170: 0.3, 180: 0.4, 190: 0.2}
P_height = {160: 0.22019458270563988, 170: 0.46938753977303344, 180: 0.15493736960539042, 190: 0.15548050791593632}
# Example probabilities of weight
# P_weight = {50: 0.2, 65: 0.3, 80: 0.4, 95: 0.1}
P_weight = {50: 0.31987029288646424, 65: 0.22918457097625072, 80: 0.17341681239768114, 95: 0.2775283237396038}
# Example conditional probabilities of sex given height and weight


sex_probabilities = calculate_sex_probability(P_height, P_weight)
print("Probability of sex:", sex_probabilities)

Probability of sex: {'M': 0.5925607183118222, 'F': 0.4074392816881778}


In [10]:
# The probability of hear disease given sex, DCM and blood type, the tuple is in this order: (heart disease, sex, blood type, DCM)
P_heart_disease_given_parents = {
    ('N', 'M', 'O', 'Y'): 0.3, ('Y', 'M', 'O', 'Y'): 0.7,
    ('N', 'M', 'A', 'Y'): 0.4, ('Y', 'M', 'A', 'Y'): 0.6,
    ('N', 'M', 'B', 'Y'): 0.5, ('Y', 'M', 'B', 'Y'): 0.5,
    ('N', 'M', 'AB', 'Y'): 0.6, ('Y', 'M', 'AB', 'Y'): 0.4,
    ('N', 'F', 'O', 'Y'): 0.6, ('Y', 'F', 'O', 'Y'): 0.4,
    ('N', 'F', 'A', 'Y'): 0.3, ('Y', 'F', 'A', 'Y'): 0.7,
    ('N', 'F', 'B', 'Y'): 0.4, ('Y', 'F', 'B', 'Y'): 0.6,
    ('N', 'F', 'AB', 'Y'): 0.7, ('Y', 'F', 'AB', 'Y'): 0.3,
    ('N', 'M', 'O', 'N'): 0.6, ('Y', 'M', 'O', 'N'): 0.4,
    ('N', 'M', 'A', 'N'): 0.15, ('Y', 'M', 'A', 'N'): 0.85,
    ('N', 'M', 'B', 'N'): 0.4, ('Y', 'M', 'B', 'N'): 0.6,
    ('N', 'M', 'AB', 'N'): 0.7, ('Y', 'M', 'AB', 'N'): 0.3,
    ('N', 'F', 'O', 'N'): 0.8, ('Y', 'F', 'O', 'N'): 0.2,
    ('N', 'F', 'A', 'N'): 0.7, ('Y', 'F', 'A', 'N'): 0.3,
    ('N', 'F', 'B', 'N'): 0.65, ('Y', 'F', 'B', 'N'): 0.35,
    ('N', 'F', 'AB', 'N'): 0.35, ('Y', 'F', 'AB', 'N'): 0.65,
}

In [11]:
def calculate_heart_disease_probability(DCM_prob: dict, sex_prob: dict, blood_type_prob: dict) -> dict:
    '''
    Function: Calculate Heart Disease Probability

    This function, `calculate_heart_disease_probability(DCM_prob, sex_prob, blood_type_prob, heart_disease_given_parents)`, computes the probability of heart disease based on the probabilities of Dilated Cardiomyopathy (DCM), sex, and blood type.

    Parameters:
    - DCM_prob: Dictionary containing probabilities for DCM categories.
    - sex_prob: Dictionary containing probabilities for sex categories.
    - blood_type_prob: Dictionary containing probabilities for blood type categories.

    Returns:
    - Dictionary containing probabilities for the presence (Y) and absence (N) of heart disease.
    '''

    heart_disease_dic = {'Y': 0, 'N': 1}

    heart_disease_probability = np.zeros(2)
    
    for d, dval in DCM_prob.items():
        for s, sval in sex_prob.items():
            for b, bval in blood_type_prob.items():
                for h, hval in heart_disease_dic.items():
                    heart_disease_probability[hval] += P_heart_disease_given_parents[(h, s, b, d)] * sval * bval * dval

    # TODO: complete the code

    return {'Y': heart_disease_probability[0], 'N': heart_disease_probability[1]}


In [12]:
# Run this code to test you implementation
# Example probabilities of DCM
# P_DCM = {'Y': 0.2, 'N': 0.8}
P_DCM = {'Y': 0.4216714899580485, 'N': 0.5783285100419515}
# Example probabilities of blood type
# P_blood_type = {'O': 0.4, 'A': 0.3, 'B': 0.2, 'AB': 0.1}
P_blood_type = {'O': 0.2753321437518179, 'A': 0.3647542741870535, 'B': 0.12749216617033832, 'AB': 0.2324214158907902}
# Example probabilities of sex
# P_sex = {'M': 0.3, 'F': 0.7}
P_sex = {'M': 0.5925607183118222, 'F': 0.4074392816881778}

heart_disease_probabilities = calculate_heart_disease_probability(P_DCM, P_sex, P_blood_type)
print("Probability of heart disease: ", heart_disease_probabilities)

Probability of heart disease:  {'Y': 0.5088821514178744, 'N': 0.4911178485821254}


# Loading DNA Sequences and Heart Disease Probability Calculation (20 points)

In this part, we will load the DNA sequences from the file DNA_patients.csv and calculate the probability of heart disease for each patient. Subsequently, we will classify the patients based on their heart disease probability and save the data.

In [13]:
# Read the CSV file into a DataFrame
df = pd.read_csv('DNA_patients.csv')

df.head()

,ID,DNA
0,1,CTCCAATACCCCCCACAAGAACACACCCATAAAATTGCAACCCACA...
1,2,TTGATGTAGAAGTATATTTGTTGGGTATTTGAGGTAACGTTATTAG...
2,3,ATATAATTTAAAGTCACTGGAAAAAAACAACCTAATAAAAACCACC...
3,4,TAAAGACAAAATTAAATTGAAGTAATGTTATGTTAAAATTTTGAAT...
4,5,AGCGTTTGTTCGTTAGCCGTAGGCAATGACGTGATTCAGGTCTGTG...


In [14]:
# TODO: complete the code
# Extract DNA sequences and store them in a list
DNAs = df['DNA']

states = ['H', 'L']
emissions = [{'A': 0.2, 'C': 0.3, 'G': 0.3, 'T': 0.2}, {'A': 0.3, 'C': 0.2, 'G': 0.2, 'T': 0.3}]
start_probabilities = [0.5, 0.5]
transition_probabilities = [{'H': 0.5, 'L': 0.5}, {'H': 0.4, 'L': 0.6}]

hmm = HMM(states, emissions, start_probabilities, transition_probabilities)

GC_content_seqs = []

for seq in DNAs:
    path, max_probability = hmm.viterbi(seq)
    GC_content_seqs.append(path)

In [15]:
# TODO: complete the code, classify the Patients as 1 if the probability of heart disease is more that 0.5 and 0 otherwise
heart_disease_classified = []

for seq in GC_content_seqs:
  h_prob, w_prob, b_prob, D_prob = probabilities_from_sequence(seq)
  s_prob = calculate_sex_probability(h_prob, w_prob)
  d_prob = calculate_heart_disease_probability(D_prob, s_prob, b_prob)
  heart_disease_classified.append(1 if d_prob['Y'] > 0.5 else 0)


In [16]:
# Write the result to CSV
labels = [(i + 1, heart_disease_classified[i]) for i in range(len(heart_disease_classified))]
csv_filename = "heart_disease_result.csv"
with open(csv_filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['ID', 'Label'])
    for id, label in labels:
        writer.writerow([id, label])

**Note: You should upload heart_disease_result.csv along side your notebook.**